<a href="https://colab.research.google.com/github/samrudhi93/Machine-Translation-techniques/blob/main/Seq2Seq_machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget /content/drive/MyDrive/Colab_Notebooks/fra.txt

/content/drive/MyDrive/Colab_Notebooks/fra.txt: Scheme missing.


In [ ]:
filename = '/content/drive/MyDrive/Colab_Notebooks/fra.txt'
txt_file = open(filename, 'r')
FileContent = txt_file.read(300)
FileContent

'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)\nGo.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)\nGo.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)\nHi.\tSalut !\tCC-BY 2.0 ('

**Data Preprocessing**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, Dense

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
FileContent

'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)\nGo.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)\nGo.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)\nHi.\tSalut !\tCC-BY 2.0 ('

#### Preparing data

In [ ]:
with open('/content/drive/My Drive/Colab_Notebooks/fra.txt', 'r') as f:
  lines = f.read().split("\n")

In [ ]:
lines

In [ ]:

#vectorise the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open('/content/drive/My Drive/Colab_Notebooks/fra.txt', 'r', encoding='utf-8') as f:
  lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")  # /t is used 2 times so for that we use '_' to split it and we get input and targets 
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    #print(input_text)
    for char in input_text:
      if char not in input_characters:
        input_characters.add(char)
    for char in target_text:
      if char not in target_characters:
        target_characters.add(char)
#Storing unique characters present in the text 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)


#Now to get the sequnece length of the text we take the max of it. the length of text is from 11 to 15. so here it take max of it which is 15
max_encoder_sequence_length = max([len(txt) for txt in input_texts]) 
max_decoder_sequence_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_sequence_length)
print("Max sequence length for outputs:", max_decoder_sequence_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(num_encoder_tokens)
input_text

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

### Data that goes into encoder decoder layer in array

In [ ]:
#Taking array of 0's and after that fillig it as per the arguments.
#MAking the structure of the array of encoder and decoder layer

encoder_input_data = np.zeros((len(input_texts), max_encoder_sequence_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_sequence_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros((len(input_texts), max_decoder_sequence_length, num_decoder_tokens), dtype="float32")


TEST code

In [ ]:
li = ["i am having best day today",
  "do you have any idea how it went",
  "Well you deserve this"]
print(len(li))
seq = max([len(txt) for txt in li])
print(seq)
token = set()
for char in li:
  if char not in token:
    token.add(char)
print(len(token))    
arr = np.zeros((len(li),seq, len(token)))
arr

In [ ]:
input_token_index["a"]

44

### One-hot encoding of all the sentences of our dataset( converting words to number)

In [ ]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    #wherever that character is present, convert it into 1.0
    encoder_input_data[i, t, input_token_index[char]] = 1.0
    #print(input_token_index)
  encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.0
    if t > 0:
    # decoder_target_data will be ahead by one timestep
    # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
  decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
  decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Now the matrix that we made encoder_input_data, decoder_target_data, decoder_inout_data is all filles with neccesary inputs. Now we give this matrix into the encoder layer and decoder layer.

### Build Model

### Setting up encoder states

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))   #shape of the inputs to be go on to lstm layer
encoder = LSTM(latent_dim, return_state=True) #Adding lstm layer
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  #the output of the encoder is the output and h(hidden_state) & c(cell_state)

# We discard `encoder_outputs` and only keep the states.
#Because the architecture of encoder does not output anything at this stage, We only maintain their hidden and cell state in the context vector,
# later on, the decoder states will derive from this encoder states.
encoder_states = [state_h, state_c]

### Setting up decoder states

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) #this initial state is the context vector of encoder side
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



### Train the model

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/100
125/125 [==============================] - 54s 404ms/step - loss: 1.1399 - accuracy: 0.7351 - val_loss: 1.0989 - val_accuracy: 0.7140
Epoch 2/100
125/125 [==============================] - 50s 401ms/step - loss: 0.8209 - accuracy: 0.7773 - val_loss: 0.8263 - val_accuracy: 0.7719
Epoch 3/100
125/125 [==============================] - 50s 397ms/step - loss: 0.6553 - accuracy: 0.8148 - val_loss: 0.6963 - val_accuracy: 0.7990
Epoch 4/100
125/125 [==============================] - 49s 396ms/step - loss: 0.5727 - accuracy: 0.8339 - val_loss: 0.6326 - val_accuracy: 0.8156
Epoch 5/100
125/125 [==============================] - 50s 399ms/step - loss: 0.5268 - accuracy: 0.8455 - val_loss: 0.5926 - val_accuracy: 0.8270
Epoch 6/100
125/125 [==============================] - 50s 398ms/step - loss: 0.4916 - accuracy: 0.8554 - val_loss: 0.5592 - val_accuracy: 0.8361
Epoch 7/100
125/125 [==============================] - 49s 396ms/step - loss: 0.4636 - accuracy: 0.8629 - val_loss: 0.5383 -

In [ ]:
# Save model
model.save("s2s")

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


### Run inference (sampling)
- encode input and retrieve initial decoder state
- run one step of decoder with this initial state and a "start of         sequence" token as target. Output will be the next target token.
- Repeat with the current target token and current states

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
from tensorflow.keras.models import load_model
model = load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
decoder_inputs = model.input[1] #input_2
decoder_state_input_h = Input(shape=(latent_dim,),name='input3')
decoder_state_input_c = Input(shape=(latent_dim,),name='input4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_sequence_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    #input_seq = input("Enter your Text: ")
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("English sentence:", input_texts[seq_index])
    print("French sentence:", decoded_sentence)


-
English sentence: Go.
French sentence: Bouge !

-
English sentence: Go.
French sentence: Bouge !

-
English sentence: Go.
French sentence: Bouge !

-
English sentence: Hi.
French sentence: Salut.

-
English sentence: Hi.
French sentence: Salut.

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run!
French sentence: Fuyons !

-
English sentence: Run.
French sentence: Fuyons !

-
English sentence: Run.
French sentence: Fuyons !

-
English sentence: Run.
French sentence: Fuyons !

-
English sentence: Run.
French sentence: Fuyons !

-
English sentence: Run.
French sentence: Fuyons !

-
English sentence: Run.
French sentence: Fuyons !

-
English sentence: Run.

### Test code


In [ ]:
a = ["hi", "how", "are", "you"]
#french
b=["salut","comment" , "ça", "va"]
c = []
for i, (x, y) in enumerate(zip(a, b)):
  print(i, (x, y))
  for t, char in enumerate(a):
    print(t, char)
    

0 ('hi', 'salut')
0 hi
1 how
2 are
3 you
1 ('how', 'comment')
0 hi
1 how
2 are
3 you
2 ('are', 'ça')
0 hi
1 how
2 are
3 you
3 ('you', 'va')
0 hi
1 how
2 are
3 you


##Test code

In [ ]:
para = 'i am recognizing the pattern in my behaviour. \thu mari swabhav ne odkhu chu. \tCC-BY 2.0 (France) Attribution: tatoeba.org #5828662 (CK) & #4678647 (sacredceltic)'

In [ ]:
sampl = 150
inp = []
out= []
input_char = set()
output_char = set()
for i in para[: min(sampl, len(para) - 1)]:
  input_tex, target_tex, _ = para.split("\t")
  target_tex = "/t" + target_tex + "\n"
  inp.append(input_tex)
  out.append(target_tex)
  for char in input_tex:
    if char not in input_char:
      input_char.add(char)
  for char in target_tex:
    if char not in output_char:
      output_char.add(char)  
      #Now we have a single set of characters stored in input_char and output_char 

input_char = sorted(list(input_char)) 
output_char = sorted(list(output_char))
max_encoder_seq_length = max([len(txt) for txt in inp])
print(max_encoder_seq_length)    

46


In [ ]:
len(inp)

150